In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, value

# Create a dictionary of the knowledge graph
knowledge_graph = {
    ('ProductA', 'Completed'): 10,
    ('ProductA', 'InProgress'): 5,
    ('ProductA', 'Pending'): 2,
    ('ProductB', 'Completed'): 8,
    ('ProductB', 'InProgress'): 7,
    ('ProductB', 'Pending'): 3,
    ('ProductC', 'Completed'): 15,
    ('ProductC', 'InProgress'): 4,
    ('ProductC', 'Pending'): 1
}

# Extract unique nodes (products) and statuses
products = list(set([edge[0] for edge in knowledge_graph.keys()]))
statuses = list(set([edge[1] for edge in knowledge_graph.keys()]))

# Create a mapping between node names and indices
product_indices = {product: i for i, product in enumerate(products)}
status_indices = {status: i for i, status in enumerate(statuses)}

# Create a matrix of the knowledge graph
matrix = csr_matrix((np.ones(len(knowledge_graph)), ([product_indices[edge[0]] for edge in knowledge_graph.keys()],
                                                     [status_indices[edge[1]] for edge in knowledge_graph.keys()])),
                    shape=(len(products), len(statuses)))

# Create a vector of the weights of the edges in the knowledge graph
weights = np.array([knowledge_graph[edge] for edge in knowledge_graph.keys()])

# Create a vector of the variables
variables = [LpVariable(f'{product}_{status}', lowBound=0) for product in products for status in statuses]

# Define the objective function (minimizing a dummy objective as we are interested in constraints)
objective = lpSum([0 * var for var in variables])  # Dummy objective

# Create the constraints
constraints = []

# Add constraint: sum of all products' statuses should equal the known weights
for i, (product, status) in enumerate(knowledge_graph.keys()):
    constraints.append(variables[i] == weights[i])

# Create the model
model = LpProblem("Product Status Query", LpMinimize)

# Add the objective function and constraints to the model
model += objective
for c in constraints:
    model += c

# Solve the model
model.solve()

# Function to query the number of completed products
def query_completed_products():
    completed_product_vars = [var for var in variables if 'Completed' in var.name]
    total_completed = sum([value(var) for var in completed_product_vars])
    return total_completed

# Query the number of completed products
completed_products = query_completed_products()
print("Number of completed products:", completed_products)

# Generate the linear equation (not strictly necessary for the query, but included for completeness)
equation = ""
for i, var in enumerate(variables):
    if value(var) > 0:
        equation += f"{value(var)} * {var.name} + "
equation = equation.rstrip(" + ") + " = Total"
print("Linear equation:", equation)


Number of completed products: 6.0
Linear equation: 10.0 * ProductB_Pending + 5.0 * ProductB_InProgress + 2.0 * ProductB_Completed + 8.0 * ProductC_Pending + 7.0 * ProductC_InProgress + 3.0 * ProductC_Completed + 15.0 * ProductA_Pending + 4.0 * ProductA_InProgress + 1.0 * ProductA_Completed = Total


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 33.7 MB/s eta 0:00:00
